In [1]:
import finn.util.platforms as p
from finn.util.platforms import Platform, DEFAULT_RES_LIMITS

class K26_Platform(Platform):
    def __init__(self):
        super().__init__()

        self.name = "K26"
        self.fpga_part = "xck26-sfvc784-2LV-c"
        self.default_clk_ns = 5.0  # 200 MHz
        self.res_limits = DEFAULT_RES_LIMITS


In [2]:
p.platforms["K26"] = K26_Platform()


In [3]:
import os, sys

print("Python:", sys.executable)
print("PWD:", os.getcwd())

try:
    import finn
    print("FINN imported from:", finn.__file__)
except Exception as e:
    print("FINN import FAILED:", e)


Python: /usr/bin/python3
PWD: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/notebooks/ellipse_regression_project_kria2
FINN imported from: None


In [4]:
import sys
print("\n".join(sys.path))


/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/tmp/home_dir/.local/lib/python3.10/site-packages
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/qonnx/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/finn-experimental/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/brevitas/src
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/deps/pyverilator
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/src
/usr/local/lib/python3.10/dist-packages
/workspace/src/dataset-loading
/usr/lib/python3/dist-packages


In [5]:
import os
assert os.path.isdir("/tools/Xilinx/Vivado"), "Vivado not mounted"
print("Vivado path OK")
print("Vivado versions:", os.listdir("/tools/Xilinx/Vivado"))

Vivado path OK
Vivado versions: ['2023.2']


In [6]:
!echo "HLS_PATH = $HLS_PATH"
!which vitis_hls
!vitis_hls -version


HLS_PATH = /tools/Xilinx/Vitis_HLS/2023.2
/tools/Xilinx/Vitis_HLS/2023.2/bin/vitis_hls
Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.2 (64-bit)
SW Build 4023990 on Oct 11 2023
IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
Tool Version Limit: 2023.10
Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.
SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023



In [7]:
from finn.builder.build_dataflow import build_dataflow_cfg
from finn.builder.build_dataflow_config import DataflowBuildConfig
from finn.util.basic import make_build_dir
from qonnx.core.modelwrapper import ModelWrapper  # <--- QONNX wrapper

print("✓ FINN imports OK")

✓ FINN imports OK


In [8]:
model_path = "ellipse_regression_hw_ready.onnx"
build_dir = make_build_dir("build_kv260")

print("Model:", model_path)
print("Build dir:", build_dir)


Model: ellipse_regression_hw_ready.onnx
Build dir: /tmp/finn_dev_hritik/build_kv260tlz4n8l7


In [9]:
model = ModelWrapper(model_path)
model.check_all_tensor_shapes_specified()
print("Model loaded successfully with QONNX wrapper")

Model loaded successfully with QONNX wrapper


In [10]:
import os
os.environ["FINN_NUM_HLS_JOBS"] = "1"


In [11]:
!python3 -c "import finn; import pkgutil; import finn.util; print('finn ok')"
!python3 -c "import finn, inspect, finn.util.platforms as p; print(dir(p))" 2>/dev/null || true
!grep -R "board=" -n "$(python3 -c 'import finn, os; print(os.path.dirname(finn.__file__))')" | head


finn ok
['Alveo_NxU200_Platform', 'Alveo_NxU250_Platform', 'Alveo_NxU280_Platform', 'Alveo_NxU50_Platform', 'Alveo_NxU55C_Platform', 'DDR_RESOURCE_REQUIREMENTS', 'DEFAULT_AVG_CONSTRAINTS', 'DEFAULT_RES_LIMITS', 'DONT_CARE', 'ETH_RESOURCE_REQUIREMENTS', 'HBM_RESOURCE_REQUIREMENTS', 'Platform', 'ZU28DR_Platform', 'ZU3EG_Platform', 'ZU7EV_Platform', 'ZU9EG_Platform', 'Zynq7020_Platform', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'abstractmethod', 'np', 'platforms']
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/posixpath.py", line 152, in dirname
    p = os.fspath(p)
TypeError: expected str, bytes or os.PathLike object, not NoneType
grep: : No such file or directory


In [24]:
import finn.util.platforms as p
from finn.util.platforms import Platform
from finn.builder.build_dataflow_config import pynq_part_map

class K26_Platform(Platform):
    def __init__(self):
        super().__init__()
        self.name = "K26"
        self.fpga_part = "xck26-sfvc784-2LV-c"
        self.default_clk_ns = 20.0
        self.res_limits = {}
        self.memories = {}

p.platforms["K26"] = K26_Platform()
pynq_part_map["K26"] = p.platforms["K26"].fpga_part


In [25]:
from finn.transformation.fpgadataflow.make_zynq_proj import pynq_native_port_width

# Assign default AXI width for K26
# 32 bits is safe for a typical Kria/Zynq dataflow
pynq_native_port_width["K26"] = 32


In [16]:
from finn.builder.build_dataflow_config import pynq_part_map
from finn.transformation.fpgadataflow.make_zynq_proj import pynq_clock_map, pynq_hbm_map

# FPGA part
pynq_part_map["K26"] = p.platforms["K26"].fpga_part

# Default clock (MHz)
pynq_clock_map["K26"] = 100

# No HBM by default
pynq_hbm_map["K26"] = []


ImportError: cannot import name 'pynq_clock_map' from 'finn.transformation.fpgadataflow.make_zynq_proj' (/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn_repo/src/finn/transformation/fpgadataflow/make_zynq_proj.py)

In [26]:
from finn.builder.build_dataflow_config import DataflowBuildConfig, DataflowOutputType, ShellFlowType

cfg = DataflowBuildConfig(
    output_dir="output_estima",
    mvau_wwidth_max=32,
    target_fps=100,
    synth_clk_period_ns=20.0,
    shell_flow_type=ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        DataflowOutputType.STITCHED_IP,
        DataflowOutputType.RTLSIM_PERFORMANCE,
        DataflowOutputType.OOC_SYNTH,
        DataflowOutputType.ESTIMATE_REPORTS,
        DataflowOutputType.BITFILE,
        DataflowOutputType.PYNQ_DRIVER,
        DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    board="KV260_SOM",
)


In [27]:
build_dataflow_cfg(model_path, cfg)


Building dataflow accelerator from ellipse_regression_hw_ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_hritik
Final outputs will be generated in output_estima
Build log is at output_estima/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16/19]
Running step: step_synthesize_bitfile [17/19]
Running ste

0

In [29]:
print("Build complete. Output files:")
for root, dirs, files in os.walk(build_dir):
    for f in files:
        if f.endswith((".bit", ".hwh")):
            print(os.path.join(root, f))

Build complete. Output files:


In [1]:
import numpy as np

# FINN expects UINT8 input with shape (1,256)
x = np.random.randint(0, 256, size=(1, 256), dtype=np.uint8)

np.save("input.npy", x)

print(x.shape, x.dtype, x.min(), x.max())


(1, 256) uint8 0 255
